In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib.ticker
from IPython.display import Markdown as md
from IPython.display import HTML, display
from time import sleep

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt

In [ ]:
min_num_for_big = 29

In [ ]:
# This file is used by web_gen.py to create a report for every FF chemical.  To make this work nicely,
# you should use the 'hide-input-all' nbextension and before running web_gen, enable hide all, save the sheet, 
#  and CLose and Halt.  

This is a script-generated report about a specific chemical used in the Hydraulic Fracking industry. The data are supplied by the industry-sponsored website [FracFocus](https://fracfocus.org/), but analyzed by the independent project [Open-FF](https://frackingchemicaldisclosure.wordpress.com/).
Structure images courtesy of [ChemID](https://chem.nlm.nih.gov/chemidplus/).

In [ ]:
# get some global values
ref_df = pd.read_csv('ref.csv')
ref_df.set_index('varname',inplace=True)


In [ ]:
md(f'This report (created on {ref_df.at["today","value"].split()[0]}) was generated from FracFocus bulk data downloaded on {ref_df.at["data_date","value"]}.')

In [ ]:
# get epa and other names
names = pd.read_csv('bgCAS.csv')
names = names.fillna('_empty_')

In [ ]:
# for nicer displays of numbers: round to significant figures.
from math import log10, floor
def round_sig(x, sig=2):
    try:
        if abs(x)>=1:
            out =  int(round(x, sig-int(floor(log10(abs(x))))-1))
            return f"{out:,d}" # does the right thing with commas
        else: # fractional numbers
            return str(round(x, sig-int(floor(log10(abs(x))))-1))
    except:
        return x

# used to translate lists or sets into a display string
def xlate_to_str(inp,sep='; ',trunc=False,tlen=20,
                 maxlen=10,maxMessage='Too many items to display'):
    try:
        l = list(inp)
        if len(l)>maxlen:
            return maxMessage
        out = ''
        for s in l:
            if trunc:
                if len(s)>tlen:
                    s = s[:tlen-3]+ '...'
            out+= s+sep
    except:
        return ''
    return out[:-(len(sep))]




In [ ]:
cas = ref_df.at['target_cas','value'].strip()
epa_sub = names[names.bgCAS==cas]['epa_Substance_Name'].values[0]
cas_orig = names[names.bgCAS==cas]['bgIngredientName'].values[0]
eh_name = names[names.bgCAS==cas]['eh_IngredientName'].values[0]
epa_reg = names[names.bgCAS==cas]['epa_Registry_Name'].values[0]

#print(f'{epa_sub}\n{epa_reg}\n{cas_orig}\n{eh_name}')

subname = epa_sub
subtitle = 'Substance name (EPA list)'
if epa_sub=='_empty_':
    subname = cas_orig
    subtitle = 'Substance name (SciFinder)'
if epa_reg != '_empty_':
    regname = epa_reg
    regtitle = 'Registry name (EPA list)'
else:
    regname = ''; regtitle = ''
if eh_name !='_empty_':
    ehname = eh_name
    ehtitle = 'Name used in Elsner/Hoelzer'
else:
    ehname = ''; ehtitle = ''

In [ ]:
# igname = passed_lines[4].strip()
# ehname = passed_lines[5].strip()
# try:
#     ehname = str(ehname)
#     if ehname=='nan': ehname=' (not classified) '
# except:
#     print('cant xlate ehname!')
    
chemid = f'<p style="text-align: center; font-size:400%">{cas}</p>'
if cas=='cas_unresolved':
    chemid = f'<p style="text-align: center; font-size:200%">{cas}</p>'

regcode = f"""       <tr>
                        <td><p style="text-align: center; font-size:100%">{regtitle}</p></td>
                    </tr>
                    <tr>
                        <td><p style="text-align: center; font-size:200%">{regname}</p>'</td>
                    </tr>"""
if regname == '':
    regcode = ''
ehcode = f"""       <tr>
                        <td><p style="text-align: center; font-size:100%">{ehtitle}</p></td>
                    </tr>
                    <tr>
                        <td><p style="text-align: center; font-size:200%">{ehname}</p>'</td>
                    </tr>"""
if ehname == '':
    ehcode = ''

display(HTML(f"""<style>
                </style>
                <table width=100% >
                <tr>
                <td>
                    <img src="https://chem.nlm.nih.gov/chemidplus/structure/{cas}" alt="no image available from ChemID"/>
                </td>
                <td>
                    <table>
                    <tr>
                        <td><p style="text-align: center; font-size:100%">CAS Registration Number</p></td>
                    </tr>
                    <tr>
                        <td>{chemid}</td>
                    </tr>
                    </table>
                <td>
                    <table>
                    <tr>
                        <td><p style="text-align: center; font-size:100%">{subtitle}</p></td>
                    </tr>
                    <tr>
                        <td><p style="text-align: center; font-size:200%">{subname}</p>'</td>
                    </tr>
                    {regcode}
                    {ehcode}
                    </table>
            </table>"""))

In [ ]:
alldf = pd.read_csv('data.csv',low_memory=False)
df = alldf[alldf.in_std_filtered].copy()
have_data = len(df)>0
have_mass = df.calcMass.max()>0

In [ ]:
if not have_data:
    display(md('## THERE ARE NO RECORDS IN THE FILTERED DATA FOR THIS CHEMICAL\n\nLimited analysis is possible\n\n'))
#     display(md('###aborting this notebook...\n\n'))
else:
    if not have_mass:
        display(md('## There are no records for this chemical for which mass is calculable.'))
        display(md('### Much of the analysis in this report will be left blank'))
    
# if cas=='cas_unresolved':
#     display(md('## The "cas_unresolved" set is a mixed bag and a mess'))
#     display(md('### aborting this notebook...\n'))
#     have_data=False
#     have_mass=False

In [ ]:
if have_data:
    df.date = pd.to_datetime(df.date)
    df['year'] = df.date.dt.year
    df = df[df.year>2010]  # FF includes some old data but before 2011 are very sparse
    display(md('Removing any disclosures before 2011; sparse and not representative.'))

### Table of Contents
- Info about the material
- [General abundance](#frequency) within FracFocus
- [Large uses](#large_uses) (top 10% by mass for chemicals with at least 30 records)
  - changes over time
  - by state
  - by operator
  - by primarySupplier
- [Detailed abundance for all records](#detailAbundance)
  - Uses by percentage of the fracking fluid
  - Uses by mass
- [Companies](#companies) supplying and using this chemical
- [Patterns of use](#patterns)
- [Disclosures list of biggest 100 uses](#biggest100) with map links
- [Tables of raw](#raw) Names, CAS Numbers and Trade names associated with this chemical

<a id='frequency'></a>
# General abundance

In [ ]:
total_Upload = int(ref_df.at['tot_num_disc_less_FFV1','value'])
fil_Upload = int(ref_df.at['tot_num_disc_fil','value'])

In [ ]:
pres = len(df.UploadKey.unique())
massPres = len(df[df.calcMass>0].UploadKey.unique())
totpres = len(alldf.UploadKey.unique())
#     display(md(f'### Number of disclosures reporting {cas}:'))


In [ ]:
if have_data:
    x = [pres,fil_Upload-pres]
    labels = [f'{cas} present','not reported']
    plt.figure(figsize=(5, 5))
    plt.pie(x,explode=[0.15,0])
    plt.legend(labels=labels)
    title = f'Percent disclosures reporting use of {cas}:'
    title += f'\n {round_sig(pres/fil_Upload*100,3)}% '
    plt.title(title,fontsize=16);
    plt.show()
    display(md(f"""||Unfiltered data set|Filtered data set|with calculated mass|
|---|---|---|---|
|Num disclosures|{totpres:,}|{pres:,}|{massPres:,}*|
|% of disclosures|{round_sig(totpres/total_Upload*100,3)}%|{round_sig(pres/fil_Upload*100,3)}%|{round_sig(massPres/fil_Upload*100,3)}%|"""))
    if massPres<pres:
        display(md(f'> *See [below](#massCompanies) for list of companies not reporting enough info for mass calculations'))

<a id='large_uses'></a>

In [ ]:

if have_mass:
    if len(df[df.calcMass>0])>min_num_for_big:
        display(md('## What are the big uses?'))
        perc90_mass = np.percentile(df[df.calcMass>0].calcMass,90)
        perc90_str = str(round_sig(perc90_mass,1))
        display(md(f'### Upper 10% of records are over {perc90_str} pounds'))
    else:
        display(md('## Not enough mass data records for analysis of big uses'))

In [ ]:
if have_mass:
    if len(df[df.calcMass>0])>min_num_for_big:
        display(md('#### ...by State'))
        df90 = df[df.calcMass>=perc90_mass].copy()
        st90 = df90.groupby('bgStateName')['bgCAS'].count()
        if len(st90)<10:
            plt.pie(st90,labels=st90.index.tolist());
        else:
            st90 = st90.reset_index()
            st90.columns = ['State Name','number of records in top 10% by mass']
            iShow(st90)


In [ ]:
if have_mass:
    if len(df[df.calcMass>0])>min_num_for_big:
        display(md('#### ...by Operator'))
        op90 = df90.groupby('bgOperatorName')['bgCAS'].count()
        if len(op90)<10:
            plt.pie(op90,labels=op90.index.tolist());
        else:
            op90 = op90.reset_index()
            op90.columns = ['Operator Name','number of records in top 10% by mass']
            iShow(op90)


In [ ]:
if have_mass:
    if len(df[df.calcMass>0])>min_num_for_big:
        display(md('#### ...by primarySupplier'))
        display(md('The primarySupplier is an indicator of the most frequent supplier for a disclosure and is not necessarily related to a given chemical record.'))
        ps90 = df90.groupby('primarySupplier')['bgCAS'].count()
        if len(ps90)<10:
            plt.pie(ps90,labels=ps90.index.tolist());
        else:
            ps90 = ps90.reset_index()
            ps90.columns = ['primarySupplier','number of records in top 10% by mass']
            iShow(ps90)


In [ ]:
if have_mass:
    if len(df[df.calcMass>0])>min_num_for_big:
        display(md('### How do large uses change over time?'))
        display(md('-- showing only top 10% by mass'))

        ax = df[df.calcMass>perc90_mass].plot('date','calcMass',#title=f'Events with >{perc90_str} lbs. {cas}',
                                            style='o',alpha=.5,
                                            figsize=(10,6))
        ax.set(yscale='log')
        ax.tick_params(axis="y", labelsize=14)
        ax.tick_params(axis="x", labelsize=14)
        #locmaj = matplotlib.ticker.LogLocator(base=10,numticks=7,subs='all') 
        locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
        ax.yaxis.set_major_locator(locmaj)
        ax.grid()        #ax.set(ylim=(10000,1000000));
        plt.ylabel(f'mass of {cas} (lbs.)',fontsize=14);
        plt.xlabel('date',fontsize=14);
        plt.title(f'Events with >{perc90_str} lbs. {cas}',fontsize=16);


<a id='detailedAbundance'></a>
# Detailed abundance
Using all **filtered** data for this chemical

### Uses by percent of the fracking job
This measure roughly shows a comparison of the concentration of the chemical in the whole fracking job (including the base fluid which is typically over 80%).  Only the disclosures where the sum of PercentHFJob is within 5% of 100% are plotted. 

In [ ]:
if have_mass:
    ax = df[df.calcMass>0].plot('date','PercentHFJob', style='o', alpha=.3,
                figsize=(10,6))
    plt.ylabel('Percent of Job',fontsize=16);
    plt.title(f'Percent of job that is {cas} - linear version',fontsize=16);
    ax.grid()
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)


In [ ]:
if have_mass:
    ax = df[df.calcMass>0].plot('date','PercentHFJob', style='o', alpha=.3,
                figsize=(10,6))
    plt.ylabel('Percent of Job',fontsize=16);
    plt.title(f'Percent of Job that is {cas} - log version',fontsize=16);
    ax.set(yscale='log')
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    #locmaj = matplotlib.ticker.LogLocator(base=10,numticks=7,subs='all') 
    locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    ax.yaxis.set_major_locator(locmaj)
    ax.set(ylim=(max(0.000001,df.PercentHFJob.min()),
                 df.PercentHFJob.max()*1.1));
    ax.grid()


<a id='by_mass'></a>
### Uses by mass
This measure shows a comparison of the absolute quantity of a chemical used in a fracking job.

In [ ]:
if have_mass:
    ax = df[df.calcMass>0].plot('date','calcMass', style='o', alpha=.3,
                figsize=(10,6))
    plt.ylabel('Mass in pounds',fontsize=16);
    plt.title(f'Mass of {cas} - linear version',fontsize=16);
    ax.grid()
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)


In [ ]:
if have_mass:
    ax = df[df.calcMass>0].plot('date','calcMass', style='o', alpha=.3,
                figsize=(10,6))
    plt.ylabel('Mass in pounds',fontsize=16);
    plt.title(f'Mass of {cas} - log version',fontsize=16);
    ax.set(yscale='log')
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    #locmaj = matplotlib.ticker.LogLocator(base=10,numticks=7,subs='all') 
    locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    ax.yaxis.set_major_locator(locmaj)
    ax.set(ylim=(max(0.01,df.calcMass.min()),df.calcMass.max()*1.2));
    ax.grid()


<a id='companies'></a>
# Companies

## Which operators are the most frequent users
The field reported below is bgOperatorName.

In [ ]:

if have_data:
    ngroups = len(df.bgOperatorName.unique())
    if ngroups < 6: fsiz = (8,3)
    else: fsiz = (8,8)
    ax = df.groupby('bgOperatorName')['UploadKey'].count()\
         .sort_values(ascending=False)[:20].plot.barh(figsize=fsiz)
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    plt.xlabel('Number of records',fontsize=16);
    plt.title(f'Number of chemical records using {cas} by operator',fontsize=16);

## Which suppliers are most frequently named for this chemical?
The field reported below is bgSupplier.

In [ ]:
if have_data:
    ngroups = len(df.bgSupplier.unique())
    if ngroups < 6: fsiz = (8,3)
    else: fsiz = (8,8)
    ax = df.groupby('bgSupplier')['UploadKey'].count()\
         .sort_values(ascending=False)[:20].plot.barh(figsize=fsiz)
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    plt.xlabel('Number of records',fontsize=16);
    plt.title(f'Number of chemical records using {cas} by Supplier',fontsize=16);


## Which primarySuppliers are most frequently associated with this chemical?
The primarySupplier is a disclosure-level value and is not necessarily related to a given chemical record. Nevertheless, it indicates the supplier with the most records in a fracking job.

In [ ]:
if have_data:
    ngroups = len(df.primarySupplier.unique())
    if ngroups < 6: fsiz = (8,3)
    else: fsiz = (8,8)
    ax = df.groupby('primarySupplier')['UploadKey'].count()\
         .sort_values(ascending=False)[:20].plot.barh(figsize=fsiz);
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    plt.xlabel('Number of records',fontsize=16);
    plt.title(f'Number of chemical records using {cas} associated with primarySupplier',fontsize=16);


## Total mass reported by operator - biggest users
The field reported below is bgOperator.

In [ ]:

if have_mass:
    ngroups = len(df[df.calcMass>0].bgOperatorName.unique())
    if ngroups < 6: fsiz = (8,3)
    else: fsiz = (8,8)
    ax =df[df.calcMass>0].groupby('bgOperatorName')['calcMass'].sum()\
         .sort_values(ascending=False)[:20].plot.barh(figsize=fsiz);
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14,rotation=75)
    plt.xlabel('Mass (pounds)',fontsize=16);

    plt.title(f'Total mass of {cas} used by operator',fontsize=16);

## Total mass reported by primarySupplier - biggest associations

In [ ]:
if have_mass:
    ngroups = len(df[df.calcMass>0].primarySupplier.unique())
    if ngroups < 6: fsiz = (8,3)
    else: fsiz = (8,8)
    ax =df.groupby('primarySupplier')['calcMass'].sum()\
         .sort_values(ascending=False)[:20].plot.barh(figsize=fsiz);
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14,rotation=75)
    plt.xlabel('Mass (pounds)',fontsize=16);
    plt.title(f'Total mass of {cas} associated with primarySupplier',fontsize=16);

<a id='patterns'></a>

In [ ]:
if have_mass:
    display(md('## Patterns of use'))
    display(md('### Across some of the larger operators'))


    sns.set(style="whitegrid")
    ops =df.groupby('bgOperatorName',as_index=False)['calcMass'].sum().sort_values(by='calcMass',
                                                                                 ascending=False)[:6].bgOperatorName.tolist()
    #print(ops)
    t = df[df.bgOperatorName.isin(ops)]
    #print(len(t))
    #t = t[t.record_flags.str.contains('M')]
    fig = plt.figure(figsize=(10,6))
    ax = sns.stripplot(t.calcMass,t.bgOperatorName,jitter=.2,alpha=.7)
    plt.xlabel(f'mass in pounds: {cas}',fontsize=14);
    plt.title(f'Variability in use of {cas} across some operators',fontsize=16)
    ax.set(xscale='log')
    ax.set(xlim=(max(0.1,t.calcMass.min()),t.calcMass.max()*1.1))
    ax.tick_params(axis="x", labelsize=14)
    ax.tick_params(axis="y", labelsize=14)
    locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    ax.xaxis.set_major_locator(locmaj)


In [ ]:
if have_mass:
    display(md('### Across some of the primarySuppliers'))

    sns.set(style="whitegrid")
    ops =df.groupby('primarySupplier',as_index=False)['calcMass'].sum().sort_values(by='calcMass',
                                                                                 ascending=False)[:6].primarySupplier.tolist()
    #print(ops)
    t = df[df.primarySupplier.isin(ops)]
    #print(len(t))
    #t = t[t.record_flags.str.contains('M')]
    fig = plt.figure(figsize=(10,6))
    ax = sns.stripplot(t.calcMass,t.primarySupplier,jitter=.2,alpha=.7)
    plt.xlabel(f'mass in pounds: {cas}',fontsize=14);
    plt.title(f'Variability in use of {cas} across some primarySuppliers',fontsize=16)
    ax.set(xscale='log')
    ax.set(xlim=(max(0.1,t.calcMass.min()),t.calcMass.max()*1.1))
    ax.tick_params(axis="x", labelsize=14)
    ax.tick_params(axis="y", labelsize=14)
    locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    ax.xaxis.set_major_locator(locmaj)


<a id='biggest100'></a>

In [ ]:
if have_mass:
    if massPres < 100:
        display(md('### All disclosures with mass'))
    else:
        display(md('### Disclosures with the 100 largest uses'))
    display(md('**calcMass** is in pounds. **map_link** provides a google map of site.  Note that the Google satellite map may be older than the well pad.'))

In [ ]:
if have_mass:
    mdf = df[df.calcMass>0].sort_values('calcMass',ascending=False).reset_index(drop=True)
    #df['mass (pounds)'] = df.calcMass.map(lambda x: round_sig(x,4))
    #df['water carrier (gallons)'] = df.TotalBaseWaterVolume.map(lambda x: round_sig(x,3))
    iShow(mdf[['calcMass','bgStateName','bgCountyName','map_link','bgOperatorName','APINumber','TotalBaseWaterVolume',
                'date','primarySupplier','bgSupplier','TradeName_trunc']].head(100))


<a id='raw'></a>
# Raw fields for this chemical
These are the fields in the raw FracFocus data as they are entered for the records positively identified as this chemical.  These raw fields often have typos, variations on a chemical name, etc.  In some cases, two variations may seem identical, but probably differ by non-printing characters.

In [ ]:

if have_data:
    display(md('## Raw CASNumber variations encountered for this chemical'))

    out = df.groupby('CASNumber',as_index=False)['bgCAS'].count().reset_index(drop=True)
    gb1 = df.groupby('CASNumber')['bgOperatorName'].apply(set).reset_index()
    gb1.columns = ['CASNumber','Operators']
    gb1.Operators = gb1.Operators.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb1,on='CASNumber',how='left')
    gb2 = df.groupby('CASNumber')['primarySupplier'].apply(set).reset_index()
    gb2.columns = ['CASNumber','primarySupplier']
    gb2.primarySupplier = gb2.primarySupplier.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb2,on='CASNumber',how='left').reset_index(drop=True)

    out.columns = ['CASNumber','Number of records','Operators','primarySuppliers']
    iShow(out)

In [ ]:
if have_data:
    display(md('## Raw IngredientName variations encountered for this chemical'))

    out = df.groupby('IngredientName',as_index=False)['bgCAS'].count().reset_index(drop=True)
    gb1 = df.groupby('IngredientName')['bgOperatorName'].apply(set).reset_index()
    gb1.columns = ['IngredientName','Operators']
    gb1.Operators = gb1.Operators.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb1,on='IngredientName',how='left')
    gb2 = df.groupby('IngredientName')['primarySupplier'].apply(set).reset_index()
    gb2.columns = ['IngredientName','primarySupplier']
    gb2.primarySupplier = gb2.primarySupplier.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb2,on='IngredientName',how='left').reset_index(drop=True)

    out.columns = ['IngredientName','Number of records','Operators','primarySuppliers']
    iShow(out)

In [ ]:
if have_data:
    display(md('## Raw TradeName variations encountered for this chemical\n Names truncated to 30 characters before processing'))

    out = df.groupby('TradeName_trunc',as_index=False)['bgCAS'].count().reset_index(drop=True)
    gb1 = df.groupby('TradeName_trunc')['bgOperatorName'].apply(set).reset_index()
    gb1.columns = ['TradeName_trunc','Operators']
    gb1.Operators = gb1.Operators.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb1,on='TradeName_trunc',how='left')
    gb2 = df.groupby('TradeName_trunc')['primarySupplier'].apply(set).reset_index()
    gb2.columns = ['TradeName_trunc','primarySupplier']
    gb2.primarySupplier = gb2.primarySupplier.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb2,on='TradeName_trunc',how='left').reset_index(drop=True)
    gb3 = df.groupby('TradeName_trunc')['bgSupplier'].apply(set).reset_index()
    gb3.columns = ['TradeName_trunc','bgSupplier']
    gb3.bgSupplier = gb3.bgSupplier.map(lambda x: xlate_to_str(x,maxlen=10))
    out = pd.merge(out,gb3,on='TradeName_trunc',how='left').reset_index(drop=True)

    out.columns = ['TradeName_trunc','Number of records','Operators','primarySuppliers','bgSupplier']
    iShow(out)

<a id='massCompanies'></a>

In [ ]:
if have_data:
    if massPres < pres:
        display(md('## Operators not reporting enough data for mass calculations for this chemical'))
        display(md('-- Shown are operators and how many records without mass, as well as the types of information missing or in error for each company'))
        tmp = df[~(df.calcMass>0)].groupby('bgOperatorName',as_index=False)['UploadKey'].count()
        df['noPerc'] = ~(df.PercentHFJob>0)
        df['noTBWV'] = ~df.has_TBWV
        df['noWaterCarrier'] = ~df.has_water_carrier
        df['outOfTolerance'] = ~df.within_total_tolerance
        gb = df[~(df.calcMass>0)].groupby('bgOperatorName',as_index=False)[['noTBWV','noWaterCarrier',
                                                                            'outOfTolerance','noPerc']].sum()
        tmp = pd.merge(tmp,gb,on='bgOperatorName',how='left').reset_index(drop=True)
        tmp.noTBWV = ((tmp.noTBWV/tmp.UploadKey)*100).round(0)
        tmp.noWaterCarrier = ((tmp.noWaterCarrier/tmp.UploadKey)*100).round(0)
        tmp.outOfTolerance = ((tmp.outOfTolerance/tmp.UploadKey)*100).round(0)
        tmp.noPerc = ((tmp.noPerc/tmp.UploadKey)*100).round(0)
        tmp.columns = ['Operator','records without mass','% without TBWV','% no water carrier record',
                       '% total percent out of tolerance','% no PercentHFJob']
        iShow(tmp)    